<a href="https://colab.research.google.com/github/agstnhrvy/UPLB-DCE-Thesis/blob/main/Thesis_Code_%5BOrganized%5D_%5BLocal_Runtime%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [3]:
# Thesis Project
# To properly install pyswmm and swmm-api to the Python file, select the python interpreter from the venv\Scripts\python.exe then install both libraries using "pip install ..."

!pip install pyswmm
!pip install swmm_api
# !pip install swmm5
!pip install openpyxl
import math
import itertools
import os, sys
# import swmm5
from pyswmm import Simulation, Links
# from pyswmm.swmm5 import SWMMException
from pyswmm.nodes import Nodes
from swmm_api import SwmmInput, SwmmOutput, SwmmReport
from swmm_api.input_file.section_labels import JUNCTIONS, CONDUITS
from swmm_api.input_file.sections import Conduit
from swmm.toolkit.shared_enum import LinkResult
# from google.colab import files
import pandas as pd
import re
import copy
import glob

# File Upload

In [4]:
# uploaded = files.upload()

inp = SwmmInput(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp")
# inp = SWMMInput("Study_Model.inp")
# available_sizes = uploaded["Available Sizes.txt"]
available_sizes = r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Available_Sizes.txt"
# pipe_prices = uploaded["Pipe_Prices.xlsx"]
# print("Uploaded files:", os.listdir())
# conduits_df = inp["CONDUITS"]
# print(conduits_df)

# Boundary Parameters

In [5]:
peak_flow_dict = {}
peak_flow_results =  {}
peak_vel = {}
adjust_pipe_size = {}
pipe_geometry = {}
conduit_ids = []
link_length = {}
in_offset = {}
out_offset = {}
vel_min = 0.8
vel_max = 5
slope_min = 0.01
geom1 = {}
geom2 = {}
geom3 = {}
geom4 = {}
trials = {}
velocity = {}
largest_size = {}
smallest_size = {}

# Data Frame

In [6]:
conduits = inp['CONDUITS']
xsections = inp['XSECTIONS']
junctions_data = inp['JUNCTIONS']
outfalls_data = inp['OUTFALLS']

# Store results
conduit_info = []

In [ ]:
for junction_name, junction_data in inp["JUNCTIONS"].items():
  junction_dict = vars(junction_data)
  print(junction_dict)

for conduit_name, conduit_data in inp["CONDUITS"].items():
  conduit_dict = vars(conduit_data) # Access all data in Conduit
  print (conduit_dict)

print(xsections)  # Print the content of the xsections InpSection object
for xsection_name, xsection_data in xsections.items():
  print(xsection_data)  # Print the name of each parameter in the xsections section

Test 1

In [ ]:
with Simulation(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp") as sim:
    links = Links(sim)  # Create a Links object
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid  # Get conduit name (link ID)
      conduit_data = link.conduit_statistics  # The 'link' object itself holds the conduit data

      peak_flow = conduit_data["peak_flow"]

      # # If you want to store it in a dictionary:
      # peak_flow_dict = {}  # Initialize an empty dictionary if needed
      # peak_flow_dict[conduit_name] = peak_flow
      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {conduit_data}")
      print(f"Peak Flow: {peak_flow}")  # Accessing a property of the conduit
      # Access other properties like link.roughness, link.inlet_offset, etc.

# Data Gathering

In [7]:
with Simulation(r"C:\Users\augustineharvey\OneDrive - University of the Philippines\Desktop\Thesis Project\SWMM Model\Study_Model.inp") as sim:
    links = Links(sim)
    for step in sim:
      pass
    for link in links:
      conduit_name = link.linkid
      conduit_data = link.conduit_statistics
      peak_flow = conduit_data["peak_flow"]

      print(f"Conduit Name: {conduit_name}")
      print(f"Data: {peak_flow}")

for conduit_name, conduit_data in inp["CONDUITS"].items():
  # Access individual conduit properties using the appropriate keys
  length = conduit_data.length  # Accessing the length property
  in_offset = conduit_data.offset_upstream # parameter for inlet offset
  out_offset = conduit_data.offset_downstream # parameter for outlet offset

  for xsection_name, xsection_data in xsections.items():
    name = xsection_name
    shape = xsection_data.shape
    geom1 = xsection_data.height # height is geom1 for most shapes
    geom2 = xsection_data.parameter_2 # parameter_2 is geom2 for most shapes
    geom3 = xsection_data.parameter_3 # parameter_3 is geom3 for most shapes
    geom4 = xsection_data.parameter_4 # parameter_4 is geom4 for most shapes
    # To get the idea of Geom 1 to Geom 4, see the guide attached in the Code Documentation in Google Docs

    print (f"Name: {name}")
    print (f"Length: {length} m")
    print (f"Shape: {shape}")
    print (f"Geometry Parameters: {geom1} m, {geom2} m, {geom3} m, {geom4} m")
    print (f"Inlet Offset: {in_offset} m")
    print (f"Outlet Offset: {out_offset} m")
    print (f"")

KeyboardInterrupt: 

# Arrange Conduits based on Pipe Flow


In [8]:
# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths


flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths

# Show results
for outfall_node, flow_path in flow_paths.items():
    print(f"All Flow Paths for {outfall_node}: {flow_path}")  # Print all paths for the outfall
    for path in flow_path:  # Iterate through each path in flow_path
        path.reverse()  # Reverse each individual path
        print(f"Flow Path: {path}")

        # Modified logic to identify conduits using node names
        conduits = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]

            # Find the conduit connecting these nodes
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits.append(conduit_name)
                    break  # Stop searching once the conduit is found

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits}")
        print(f"Junctions in Path: {junctions}")

All Flow Paths for OF-X1: [['OF-X1', 'X4', 'X3', 'X2', 'X1']]
Flow Path: ['X1', 'X2', 'X3', 'X4', 'OF-X1']
Sorted Conduits: ['44', '45', '46', '91']
Junctions in Path: ['X1', 'X2', 'X3', 'X4', 'OF-X1']
All Flow Paths for OF-X2: [['OF-X2', 'X26', 'X25', 'X24', 'X23', 'X22', 'X21', 'X21/2', 'X21/1']]
Flow Path: ['X21/1', 'X21/2', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'OF-X2']
Sorted Conduits: ['39', '386', '387', '388', '40', '41', '42', '43']
Junctions in Path: ['X21/1', 'X21/2', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'OF-X2']
All Flow Paths for OF-W1: [['OF-W1', 'W16', 'W15', 'W14', 'W13', 'W12', 'W11', 'W10', 'W9', 'W8', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1']]
Flow Path: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16', 'OF-W1']
Sorted Conduits: ['75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '93']
Junctions in Path: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', '

# Identify the Invert Elevations of the Conduits

In [9]:
conduit_info = []
for conduit_name, conduit_data in inp['CONDUITS'].items():
    upstream_junction = conduit_data.from_node
    downstream_junction = conduit_data.to_node

    upstream_invert = junctions_data[upstream_junction].elevation

    # Check if downstream node is an outfall
    if downstream_junction in outfalls_data:
        downstream_invert = outfalls_data[downstream_junction].elevation  # Get elevation from outfalls_data
    elif downstream_junction in junctions_data:
        downstream_invert = junctions_data[downstream_junction].elevation  # Get elevation from junctions_data if it's a junction
    else:
        print(f"Warning: Downstream node '{downstream_junction}' not found in junctions or outfalls data.")
        downstream_invert = None  # or some default value

    length = conduit_data.length

    if downstream_invert is not None:
        slope = (upstream_invert - downstream_invert) / length
    else:
        slope = None
    # Input all data gathered and solve in a DataFrame
    conduit_info.append([conduit_name, upstream_junction, upstream_invert, downstream_junction, downstream_invert, length, slope])

    print(f"Conduit: {conduit_name}; From: {upstream_junction} To: {downstream_junction}")
    print(f"Upstream Invert Elevation: {upstream_invert} m")
    print(f"Downstream Invert Elevation: {downstream_invert} m")
    print(f"Slope: {slope}")

# Exporting DataFrame as an .xlsx File
conduit_df = pd.DataFrame(conduit_info, columns=['Conduit', 'From Node', 'Upstream Invert', 'To Node', 'Downstream Invert', 'Length', 'Slope'])

conduit_df.to_excel("conduit_analysis_data.xlsx", index=False)



Conduit: 39; From: X21/1 To: X21/2
Upstream Invert Elevation: 22.33 m
Downstream Invert Elevation: 22.2 m
Slope: 0.006595636732622983
Conduit: 40; From: X23 To: X24
Upstream Invert Elevation: 21.05 m
Downstream Invert Elevation: 21.01 m
Slope: 0.0020090406830737895
Conduit: 41; From: X24 To: X25
Upstream Invert Elevation: 21.01 m
Downstream Invert Elevation: 20.97 m
Slope: 0.002000000000000135
Conduit: 42; From: X25 To: X26
Upstream Invert Elevation: 20.97 m
Downstream Invert Elevation: 20.93 m
Slope: 0.002352941176470538
Conduit: 44; From: X1 To: X2
Upstream Invert Elevation: 22.33 m
Downstream Invert Elevation: 22.14 m
Slope: 0.009476309226932555
Conduit: 45; From: X2 To: X3
Upstream Invert Elevation: 22.14 m
Downstream Invert Elevation: 22.08 m
Slope: 0.0030000000000001137
Conduit: 46; From: X3 To: X4
Upstream Invert Elevation: 22.08 m
Downstream Invert Elevation: 22.03 m
Slope: 0.0029411764705880683
Conduit: 51; From: T1 To: T2
Upstream Invert Elevation: 22.14 m
Downstream Invert E

# Function: Identifying Peak Flow Rates

In [11]:
def peak_flow_solve(inp, conduit_name):
  with Simulation(inp) as sim:
      links = Links(sim)
      for step in sim:
          pass

      # Iterate through links to find the specific conduit
      for link in links:
          if link.linkid == conduit_name:
              conduit_data = link.conduit_statistics
              peak_flow = conduit_data.get("peak_flow")  # Use get() to handle missing keys
              if peak_flow is not None and peak_flow != 0.0:
                  return float(peak_flow)  # Convert to float if found
              else:
                  return 0.0  # Or handle the missing peak_flow case differently

      # If the conduit is not found
      return 0.0  # Or handle the missing conduit case differently

# Function: Identifying the Largest Pipe Size and Minimum Slope

In [12]:
# Identify the largest pipe size available and the minimum slope possible
def largest_size_available(available_sizes):
  largest_size = 0.0
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size > largest_size:
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return largest_size

largest_size = largest_size_available(available_sizes)
print(f"Largest Pipe Size: {largest_size} m")

Largest Pipe Size: 1.8 m


# Function: Identifying the Smallest Pipe Size

In [13]:
# Identify the smallest pipe size available
def smallest_size_available(available_sizes):
  largest_size = largest_size_available(available_sizes)
  smallest_size = largest_size
  with open(available_sizes) as file:
    for line in file:
      try:
        size = float(line.strip())
        if size < largest_size:
          smallest_size = size
          largest_size = size
      except ValueError:
        print (f"Skipping Invalid Line: {line.strip()}")
  return smallest_size

# Call the function and assign the result to a variable
smallest_size = smallest_size_available(available_sizes)

print(f"Smallest Pipe Size: {smallest_size} m")

Smallest Pipe Size: 0.3 m


# Function: Identifying Suitable Pipe Sizes

In [14]:
def find_suitable_sizes(conduit_name, available_sizes, peak_flow):
  """Finds suitable pipe sizes based on peak flow and velocity constraints."""
  if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
    original_size = inp["XSECTIONS"][conduit_name].height

    if peak_flow == 0:
      return [original_size]  # Return original size if peak flow is 0

    available_sizes_list = []
    with open(available_sizes) as file:
      for line in file:
        try:
          size = float(line.strip())
          available_sizes_list.append(size)
        except ValueError:
          print(f"Skipping invalid line: {line.strip()}")

    suitable_sizes = []
    for size in available_sizes_list:
      # Calculate velocity for the current size
      velocity = peak_flow / (math.pi * (size ** 2) * 0.25)
      print(f"Trying size {size} for {conduit_name}, velocity: {velocity}")  # Print size and velocity

      # Check if velocity is within the desired range
      if vel_min <= velocity <= vel_max:
        suitable_sizes.append(size)
      else:
        # Adjust size based on velocity constraints
        if velocity < vel_min:
          # Find the closest larger size to meet minimum velocity
          closest_larger_size = min(filter(lambda x: x > size, available_sizes_list), default=None)
          if closest_larger_size:
            suitable_sizes.append(closest_larger_size)
        elif velocity > vel_max:
          # Find the closest smaller size to meet maximum velocity
          closest_smaller_size = max(filter(lambda x: x < size, available_sizes_list), default=None)
          if closest_smaller_size:
            suitable_sizes.append(closest_smaller_size)

    if not suitable_sizes:
      # If no suitable sizes found within velocity range
      # adjust to find a closest size to maintain constraints
      if velocity < vel_min:
          diameter = abs(math.sqrt((peak_flow * 4) / (math.pi * vel_min)))
      elif velocity > vel_max:
          diameter = abs(math.sqrt((peak_flow * 4) / (math.pi * vel_max)))

      closest_size = min(available_sizes_list, key=lambda x: abs(x - diameter))
      suitable_sizes.append(closest_size)

    return suitable_sizes  # Return the list of suitable sizes

# Function: Construction Cost

In [ ]:
# Function for the Computation of Construction Costs
def construction_costs (unit_cost, length, size):
  pass

# Function: Solving for the Max Hydraulic Radius

In [15]:
def get_max_hydraulic_radius(inp, conduit_name):

    with Simulation(inp) as sim:
        links = Links(sim)
        max_hydraulic_radius = 0.0  # Initialize max hydraulic radius

        for step in sim:
            # Iterate through links to find the specific conduit
            for link in links:
                if link.linkid == conduit_name:
                    # Get the flow and depth for the current link and timestep
                    flow = link.flow
                    depth = link.depth

                    # Calculate the hydraulic radius (R = A/P)
                    # Assume circular pipe for this example
                    if inp["XSECTIONS"][conduit_name].shape == "CIRCULAR":
                        diameter = inp["XSECTIONS"][conduit_name].height

                        # Calculate area and wetted perimeter based on depth
                        if depth >= diameter:  # Full flow
                            area = math.pi * (diameter / 2)**2
                            wetted_perimeter = math.pi * diameter
                        else:  # Partial flow
                            theta = 2 * math.acos(1 - (2 * depth / diameter))
                            area = (diameter**2 / 8) * (theta - math.sin(theta))
                            wetted_perimeter = diameter * theta / 2

                        hydraulic_radius = area / wetted_perimeter

                        # Update max_hydraulic_radius if current value is larger
                        max_hydraulic_radius = max(max_hydraulic_radius, hydraulic_radius)

                    else:
                        # Handle other cross-section shapes if needed
                        pass
                    break  # Exit the inner loop once the conduit is found

    return max_hydraulic_radius

# Function: Manning's Equation

In [16]:
def adjust_slope_with_mannings(input, conduit_name, new_diameter, mannings_n):

    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    # Get peak flow rate (Q) for the conduit - you'll need to adapt this part
    # based on how you are getting peak flow rates in your code
    # peak_flow = peak_flow_solve(input_file, conduit_name) # or use your similar function
    # peak_flow = get_peak_flow(modified_inp, conduit_name)
    # peak_flow = current_peak_flows[conduit_name]

    # Calculate cross-sectional area and hydraulic radius for the new diameter
    area = math.pi * (new_diameter / 2)**2
    hydraulic_radius = (new_diameter / 2) / 2  # Assuming full flow for now

    # Calculate the new slope (S) using Manning's equation
    new_slope = (peak_flow * mannings_n / (area * hydraulic_radius**(2/3)))**2

    # Return the modified SwmmInput object
    return new_slope

# Function: Identifying Slopes

In [17]:
def adjusting_slopes(modified_inp, conduit_name, pipe_size, is_last_conduit):

    # Get upstream and downstream junction names for the conduit
    upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
    downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

    # Handle the last conduit differently
    if is_last_conduit:
        # Use original downstream elevation from OUTFALLS section
        new_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
    else:
        # Otherwise, get downstream elevation from JUNCTIONS section
        new_downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

    # Calculate the new upstream invert elevation using Manning's equation and the optimized size
    new_slope = adjust_slope_with_mannings(modified_inp, conduit_name, pipe_size)
    conduit_length = modified_inp["CONDUITS"][conduit_name].length
    new_upstream_elevation = new_downstream_elevation + (new_slope * conduit_length)

    return new_upstream_elevation

# Step 1: Initialize All Pipe Sizes and Slopes

In [ ]:
modified_inp = copy.deepcopy(inp)

# Get network topology
conduit_connections = {}
for conduit_name, conduit_data in inp['CONDUITS'].items():
    conduit_connections[conduit_name] = {
        'upstream': conduit_data.from_node,
        'downstream': conduit_data.to_node
    }

# Get all outfall node names from the SWMM input file
outfall_node_names = list(inp['OUTFALLS'].keys())

def is_outfall_node(node):
    """Checks if a node is an outfall node."""
    return node in outfall_node_names  # Direct comparison with outfall node names

def trace_upstream_flow(start_node, path=None, all_paths=None):
    """Traces all upstream flow paths from a given node, including junctions and branches."""
    if path is None:
        path = []
    if all_paths is None:
        all_paths = []

    path.append(start_node)

    # Find connected conduits
    connected_conduits = [
        conduit_name
        for conduit_name, data in conduit_connections.items()
        if data['downstream'] == start_node
    ]

    # Base case: outermost junction or no upstream conduits
    if not connected_conduits:
        all_paths.append(path.copy())  # Add a copy of the path to avoid modification
        return

    # Recursive case: trace upstream for each connected conduit
    for conduit_name in connected_conduits:
        upstream_node = conduit_connections[conduit_name]['upstream']
        trace_upstream_flow(upstream_node, path.copy(), all_paths)  # Pass a copy of path to avoid modifying the original path

    return all_paths

flow_paths = {}
for outfall_node_name in outfall_node_names:
    all_paths = trace_upstream_flow(outfall_node_name)
    flow_paths[outfall_node_name] = all_paths


# Slope adjustment logic integrated with flow path identification
for outfall_node, flow_path in flow_paths.items():
    print(f"Processing outfall node: {outfall_node}")
    for path in flow_path:
        path.reverse()  # Reverse each individual path for upstream to downstream processing

        # Identify conduits in the path
        conduits_in_path = []
        for i in range(len(path) - 1):
            from_node = path[i]
            to_node = path[i + 1]
            for conduit_name, conduit_data in conduit_connections.items():
                if conduit_data['upstream'] == from_node and conduit_data['downstream'] == to_node:
                    conduits_in_path.append(conduit_name)
                    break

        junctions = [item for item in path if isinstance(item, str) and item not in conduit_connections]
        print(f"Sorted Conduits: {conduits_in_path}")  # Print sorted conduits
        print(f"Junctions in Path: {junctions}")  # Print junctions in path

        # Reverse the conduits_in_path list before processing
        conduits_in_path.reverse()

        current_downstream_elevation = None  # Initialize for the first (last) conduit

        for conduit_name in conduits_in_path:  # Iterate in the correct order

            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

            # Always use slope_min, regardless of original slope
            slope = slope_min

            # Find the xsection associated with this conduit
            xsection_name = None
            for xs_name, xs_data in xsections.items():
                if xs_data.link == conduit_name:
                    xsection_name = xs_name
                    break

            # Assign the largest size if xsection is found
            if xsection_name is not None:
                xsection_data = modified_inp["XSECTIONS"][xsection_name]

                if callable(largest_size_available):
                    conduit_size = largest_size_available(available_sizes)
                else:
                    conduit_size = largest_size_available

                if xsection_data.shape == "CIRCULAR":
                    xsection_data.height = conduit_size

            # If it's the first conduit (last in the original path),
            # use original downstream elevation from OUTFALLS or JUNCTIONS
            if conduit_name == conduits_in_path[0]:
                if downstream_junction in modified_inp["OUTFALLS"]:
                    current_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
                else:
                    current_downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

            # Otherwise, use the calculated upstream elevation of the previous conduit
            # as the current downstream elevation
            else:
                current_downstream_elevation = new_upstream_elevation  # Use the previously calculated value

            # Calculate new upstream elevation based on slope and current downstream elevation
            new_upstream_elevation = current_downstream_elevation + (slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            modified_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            print(f"Name: {xsection_name}")
            print(f"Shape: {xsection_data.shape}")
            print(f"Conduit Height: {xsection_data.height}")
            print(f"Downstream Elevation: {current_downstream_elevation}")
            print(f"Upstream Elevation: {new_upstream_elevation}")
            print(f"")

# Create a unique filename to avoid overwriting
file_name = f"Modified_Model1.inp"
modified_inp.write_file(file_name)

# Step 2: Optimize Pipe Sizes

Starting Pipe Size Optimization

In [1]:
# Get all starting pipes from all flow paths (ensure uniqueness)
all_starting_pipes = []
for outfall_node, flow_path in flow_paths.items():
    print(f"Outfall Node: {outfall_node}, Flow Path: {flow_path}")  # Print the flow path for each outfall node
    for path in flow_path:
        print(f"Path: {path}")  # Print each individual path
        conduits_in_path = [item for item in path if isinstance(item, str) and item in conduit_connections]
        print(f"Conduits in Path: {conduits_in_path}")  # Print the identified conduits in the path

        # Check if any conduits were found in this path and it's not empty
        if conduits_in_path and conduits_in_path[0]:
            starting_pipe = conduits_in_path[0]  # Get the first conduit as the starting pipe
            if starting_pipe not in all_starting_pipes:  # Check for duplicates
                all_starting_pipes.append(starting_pipe)

print(f"All Starting Pipes: {all_starting_pipes}")  # Print the final list of starting pipes

# Get suitable sizes for starting pipes ONLY
suitable_sizes_per_pipe = {}
for conduit_name in all_starting_pipes:
    # Ensure conduit_name is a string
    conduit_name_str = str(conduit_name)

    # Get peak flow for the conduit
    peak_flow_value = peak_flow_solve("Modified_Model1.inp", conduit_name_str)

    # Get suitable sizes based on the peak flow value
    sizes = find_suitable_sizes(conduit_name_str, available_sizes, peak_flow_value)

    # If suitable sizes are found, store them; otherwise, handle it
    if sizes:
        suitable_sizes_per_pipe[conduit_name_str] = set(sizes) # Convert to set for uniqueness
    else:
        # Get the original size if no suitable sizes were found
        original_size = inp["XSECTIONS"].get(conduit_name_str, {}).get('height')
        if original_size is not None:
            suitable_sizes_per_pipe[conduit_name_str] = {original_size}  # Store original size as the only option
        else:
            print(f"Error: Could not find original size for conduit '{conduit_name_str}'. Skipping.")

# Optimization: Create pipe-to-xsection mapping
pipe_to_xsection = {}
for pipe in all_starting_pipes:  # Iterate through starting pipes only
    for xs_name, xs_data in xsections.items():
        if xs_data.link == pipe:
            pipe_to_xsection[pipe] = xs_name
            break

# Instead of creating a list, use itertools.product directly in the loop
for size_combination in itertools.product(*[suitable_sizes_per_pipe[pipe] for pipe in all_starting_pipes]):
    modified_inp = copy.deepcopy(inp)  # Create a copy of the original input

    # Modify pipe sizes for all starting pipes
    for i, pipe in enumerate(all_starting_pipes):
        xsection_name = pipe_to_xsection.get(pipe)
        if xsection_name is not None:
            modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

    # Save the modified input file
    file_name = f"modified_input_starting_pipes_{'_'.join(map(str, size_combination))}.inp"

NameError: name 'flow_paths' is not defined

Downstream Optimization

In [ ]:
# --- Iterate downstream and optimize remaining conduits ---
optimized_input_files = glob.glob("modified_input_starting_pipes_*.inp")

    # Downstream Optimization
for input_file in optimized_input_files:
    modified_inp = SwmmInput(input_file)  # Load input file

    # --- Downstream Optimization Logic ---
    current_peak_flows = {}
    for conduit_name, _ in modified_inp["CONDUITS"].items():
        peak_flow_value = peak_flow_solve(input_file, conduit_name)  # Use input_file here
        current_peak_flows[conduit_name] = peak_flow_value

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Get suitable sizes for all downstream conduits in the path
        suitable_sizes_per_conduit = {}
        for conduit_name in conduits_in_path:
            # Skip starting pipes as they are already set
            if conduit_name in all_starting_pipes:
                continue
            suitable_sizes_per_conduit[conduit_name] = find_suitable_sizes(conduit_name, available_sizes, current_peak_flows[conduit_name])

        # Generate all size combinations for downstream conduits
        all_size_combinations = list(itertools.product(*[suitable_sizes_per_conduit[pipe] for pipe in suitable_sizes_per_conduit]))

        # Iterate through each size combination
        for size_combination in all_size_combinations:
            combination_modified_inp = copy.deepcopy(modified_inp)  # Create a copy for each combination

            # Apply the size combination to the input file
            for i, conduit_name in enumerate(suitable_sizes_per_conduit):
                # Find the xsection name associated with the conduit
                xsection_name = None
                for xs_name, xs_data in xsections.items():
                    if xs_data.link == conduit_name:
                        xsection_name = xs_name
                        break

                # Update the modified input with the current size from the combination
                if xsection_name is not None:
                    combination_modified_inp["XSECTIONS"][xsection_name].height = size_combination[i]

            # Save the modified input file for this combination
            file_name = f"optimized_{input_file.replace('.inp', '')}_combination_{'_'.join(map(str, size_combination))}.inp"
            combination_modified_inp.write_file(file_name)
            print(f"Optimized input file saved as {file_name}")

# Step 3: Adjust Pipe Slopes Based on Adjusted Pipe Sizes

In [ ]:
# Get a list of all optimized input files from Step 2
optimized_input_files = glob.glob("optimized_modified_input_starting_pipes_*_combination_*.inp")

# Iterate through each optimized input file
for input_file in optimized_input_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Process conduits in reverse order (from last to first)
        current_downstream_elevation = None  # Initialize for the first (last) conduit

        for i, conduit_name in enumerate(reversed(conduits_in_path)):
            # Get optimized size
            optimized_size = modified_inp["XSECTIONS"][xsection_name].height

            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node
            roughness = modified_inp["CONDUITS"][conduit_name].roughness
            # If it's the last conduit, use original downstream elevation from OUTFALLS
            if i == 0:  # i = 0 for the last conduit in reversed order
                current_downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation

            # Otherwise, use the upstream elevation of the previous conduit
            # as the current downstream elevation


            # Calculate new slope using Manning's equation and max hydraulic radius
            new_slope = adjust_slope_with_mannings(modified_inp, conduit_name, optimized_size, roughness)

            # Calculate new upstream elevation
            new_upstream_elevation = current_downstream_elevation + (new_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation
            modified_inp["JUNCTIONS"][upstream_junction].elevation = new_upstream_elevation

            # Update current_downstream_elevation for the next conduit in the loop
            current_downstream_elevation = new_upstream_elevation

    # Save the modified input file with a new name
    modified_inp.write_file(f"adjusted_{input_file}")
    print(f"Adjusted slopes/inverts for {input_file} and saved as adjusted_{input_file}")

# Step 4: Optimize Pipe Slopes

In [ ]:
# Define maximum and minimum allowable slopes (adjust values as needed)
max_slope = 0.02  # Maximum allowable slope
min_slope = 0.001  # Minimum allowable slope

# Load ground slope data from "Ground Slopes.xlsx"
ground_slopes_df = pd.read_excel("Ground_Slopes.xlsx")

# Get a list of all adjusted input files from Step 3
adjusted_input_files = [f for f in os.listdir() if f.startswith("adjusted_optimized_input_starting_pipes_")]

# Iterate through each adjusted input file
for input_file in adjusted_input_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    for outfall_node, flow_path in flow_paths.items():
        conduits_in_path = [item for item in flow_path if item in conduit_connections]

        # Process conduits in reverse order (from last to first)
        for i, conduit_name in enumerate(reversed(conduits_in_path)):
            # Get upstream and downstream junction names
            upstream_junction = modified_inp["CONDUITS"][conduit_name].from_node
            downstream_junction = modified_inp["CONDUITS"][conduit_name].to_node

            # Get junction data for upstream and downstream junctions
            upstream_junction_data = modified_inp["JUNCTIONS"][upstream_junction]
            downstream_junction_data = modified_inp["JUNCTIONS"][downstream_junction]

            # Calculate ground slope using junction data
            upstream_elev = upstream_junction_data.elevation + upstream_junction_data.depth_max
            downstream_elev = downstream_junction_data.elevation + downstream_junction_data.depth_max
            length = modified_inp["CONDUITS"][conduit_name].length
            ground_slope = (upstream_elev - downstream_elev) / length

            # Apply slope adjustment logic
            if ground_slope >= 0:
                if ground_slope <= max_slope:
                    final_slope = ground_slope  # Use ground slope if within limits
                else:
                    final_slope = max_slope  # Use max slope if ground slope is too steep
            else:
                final_slope = min_slope  # Use min slope if ground slope is negative

            # If it's the last conduit, use original downstream elevation from OUTFALLS
            if i == 0:
                downstream_elevation = modified_inp["OUTFALLS"][downstream_junction].elevation
            else:
                downstream_elevation = modified_inp["JUNCTIONS"][downstream_junction].elevation

            # Calculate new upstream elevation based on final slope
            upstream_elevation = downstream_elevation + (final_slope * modified_inp["CONDUITS"][conduit_name].length)

            # Update upstream junction elevation in the modified inp file
            modified_inp["JUNCTIONS"][upstream_junction].elevation = upstream_elevation

    # Save the modified input file with a new name
    modified_inp.write_file(f"slope_optimized_{input_file}")
    print(f"Slopes optimized for {input_file} and saved as slope_optimized_{input_file}")

# Step 5: Solving for Construction Costs

In [ ]:
# Load pipe prices data (assuming the file is named "pipe_prices.xlsx")
pipe_prices_df = pd.read_excel("Pipe_Prices.xlsx")

# Get a list of all slope-optimized input files from Step 4
slope_optimized_files = [f for f in os.listdir() if f.startswith("slope_optimized_")]

# Initialize variables to store the least cost and corresponding file name
least_cost = float('inf')  # Initialize with a very large value
least_cost_file = None

# Iterate through each slope-optimized input file
for input_file in slope_optimized_files:
    # Load the input file using SwmmInput
    modified_inp = SwmmInput(input_file)

    total_cost = 0  # Initialize total cost for this input file

    # Iterate through conduits in the input file
    for conduit_name, conduit_data in modified_inp["CONDUITS"].items():
        # Get conduit diameter (assuming it's stored in the 'height' attribute)
        diameter = conduit_data.height

        # Get conduit length
        length = conduit_data.length

        # Get pipe price per length for the current diameter from the pipe_prices_df
        price_per_length = pipe_prices_df[pipe_prices_df["Diameter"] == diameter]["Price per Length"].values[0]

        # Calculate cost for this conduit and add to total cost
        conduit_cost = price_per_length * length
        total_cost += conduit_cost

        # Check if this file has the least cost so far
        if total_cost < least_cost:
          least_cost = total_cost
          least_cost_file = input_file
    # Print total cost for this input file
    print(f"Total construction cost for {input_file}: {total_cost}")

# Print the file with the least cost
print(f"\nFile with the least construction cost: {least_cost_file} (Cost: {least_cost})")

# Sensitivity Analysis


In [ ]:
# Get the file with the least construction cost from Step 5
least_cost_file = "slope_optimized_optimized_modified_input_starting_pipes_0.6_0.45_combination_0.3_0.3_0.45_0.3_0.3_0.3.inp" # Replace with your actual file name

# Load the least cost input file
least_cost_inp = SwmmInput(least_cost_file)

# Function to calculate average flow rate, size, and slope
def get_average_values(inp_file):
    """Calculates average flow rate, size, and slope for all pipes."""
    total_flow = 0
    total_size = 0
    total_slope = 0
    num_pipes = 0

    for conduit_name, conduit_data in inp_file["CONDUITS"].items():
        for xsection_name, xsection_data in inp["XSECTIONS"].items():
        # Calculate slope
          upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
          downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
          length = conduit_data.length
          slope = (upstream_invert - downstream_invert) / length

          # Get peak flow using peak_flow_solve
          peak_flow = peak_flow_solve(inp_file, conduit_name)

          # Get pipe size (assuming it's stored in the 'height' attribute)
          size = conduit_data.height

          total_flow += peak_flow
          total_size += size
          total_slope += slope
          num_pipes += 1

    avg_flow = total_flow / num_pipes if num_pipes else 0
    avg_size = total_size / num_pipes if num_pipes else 0
    avg_slope = total_slope / num_pipes if num_pipes else 0

    return avg_flow, avg_size, avg_slope

# Get average values for the least cost file
avg_flow, avg_size, avg_slope = get_average_values(least_cost_inp)
print(f"Average Flow Rate: {avg_flow}")
print(f"Average Pipe Size: {avg_size}")
print(f"Average Slope: {avg_slope}")

# Choose a pipe for sensitivity analysis (you can modify this)
target_pipe = "C1"  # Replace with the actual pipe name

# Function to get flow rate, size, and slope for a specific pipe
def get_pipe_values(inp_file, pipe_name):
    """Gets flow rate, size, and slope for a specific pipe."""
    conduit_data = inp_file["CONDUITS"][pipe_name]

    # Calculate slope
    upstream_invert = inp_file["JUNCTIONS"][conduit_data.from_node].elevation
    downstream_invert = inp_file["JUNCTIONS"][conduit_data.to_node].elevation
    length = conduit_data.length
    slope = (upstream_invert - downstream_invert) / length

    peak_flow = peak_flow_solve(inp_file, pipe_name) # peak_flow for the chosen conduit
    size = conduit_data.height

    return peak_flow, size, slope

# Get initial values for the target pipe
initial_flow, initial_size, initial_slope = get_pipe_values(least_cost_inp, target_pipe)
print(f"Flow Rate of {target_pipe}: {initial_flow}")
print(f"Size of {target_pipe}: {initial_size}")
print(f"Slope of {target_pipe}: {initial_slope}")


# # Define rainfall scenarios (replace with your actual scenarios)
# rainfall_scenarios = ["baseline", "scenario1", "scenario2"]

# # Sensitivity analysis loop for rainfall scenarios
# for scenario in rainfall_scenarios:
#     # Load modified input file for the current scenario
#     modified_inp_file = f"{scenario}.inp"  # Assuming you have .inp files for each scenario

#     # Create a copy of the least cost input file and modify with the given rainfall intensity
#     modified_inp = copy.deepcopy(least_cost_inp) # modified_inp is now the least cost input

#     # Placeholder - Update rainfall data in the copied least_cost_inp
#     # You need to implement your logic here to update the [TIMESERIES] section in modified_inp with data for the current scenario (modified_inp_file)

#     modified_inp.write_file("modified_least_cost.inp") # write modified inp to a temporary file to check the current peak flows
#     modified_inp = SwmmInput("modified_least_cost.inp") # call this so that the swmm library will update its parameters as well


#     # Get values for the target pipe after applying the rainfall scenario
#     flow, size, slope = get_pipe_values(modified_inp, target_pipe)

#     # Calculate the change in flow rate
#     flow_change = flow - initial_flow

#     # Print the results
#     print(f"\nScenario: {scenario}")
#     print(f"Flow Rate Change for {target_pipe}: {flow_change}")
#     print(f"Flow Rate of {target_pipe}: {flow}")
#     print(f"Size of {target_pipe}: {size}")
#     print(f"Slope of {target_pipe}: {slope}")